## ENVIRONMENT SETUP

In [1]:
!nvidia-smi

Sun Apr  9 13:22:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/allenai/scirepeval.git
%cd scirepeval
!pip install --upgrade pip
!pip install -q -r requirements.txt

Cloning into 'scirepeval'...
remote: Enumerating objects: 1500, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 1500 (delta 239), reused 263 (delta 204), pack-reused 1066
Receiving objects: 100% (1500/1500), 14.17 MiB | 27.80 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
/content/scirepeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.1 MB/s

In [3]:
# Needed to fix some version conflict bug with protobuf
!pip install --upgrade protobuf
!cp /usr/local/lib/python3.9/dist-packages/google/protobuf/internal/builder.py /content/scirepeval
!pip install protobuf==3.19.5
!mv /content/scirepeval/builder.py /usr/local/lib/python3.9/dist-packages/google/protobuf/internal

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.10.0 which is incompatible.
tensorflow-metadata 1.13.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.
tensorboardx 2.5.1 requires protobuf<=3.20.1,>=3.8.0, but you have protobuf 4.22.1 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.
grpcio-status 1.48.2 requires grpcio>=1.48.2, but you have 

## TRAINING

- **SKIP THIS PART** if you only want to try out inferencing !!

### Dataset Creation and Preprocessing



In [9]:
import datasets

fos = datasets.load_dataset('allenai/scirepeval', 'fos')

{'train': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/fos/train.jsonl', 'val': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/fos/val.jsonl', 'test': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/test/fos/meta.jsonl'}


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating evaluation split:   0%|          | 0/68147 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/541218 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/67631 [00:00<?, ? examples/s]

Dataset scirepeval downloaded and prepared to /root/.cache/huggingface/datasets/allenai___scirepeval/fos/1.1.0/05b6b341c1750891a5539df5b3eb892babd18e8f299bfaba4735b87e20f6cf83. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
fos

DatasetDict({
    evaluation: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 68147
    })
    train: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 541218
    })
    validation: Dataset({
        features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
        num_rows: 67631
    })
})

In [11]:
# Finding no. of unique labels
import itertools

labels = list(itertools.chain.from_iterable(fos['train']['labels']))
label_texts = list(itertools.chain.from_iterable(fos['train']['labels_text']))

labels_dict = dict()
temp = set()

for label_text, label in zip(label_texts, labels):
    temp.add((label_text, label))

labels_dict = {label_text:label for label_text, label in list(temp)}
num_labels = len(labels_dict)

print(labels_dict)

{'Linguistics': 14, 'History': 12, 'Chemistry': 4, 'Philosophy': 18, 'Physics': 19, 'Geography': 10, 'Business': 3, 'Sociology': 22, 'Economics': 6, 'Geology': 11, 'Education': 7, 'Law': 13, 'Computer science': 5, 'Environmental science': 9, 'Agricultural and Food sciences': 0, 'Materials science': 15, 'Political science': 20, 'Medicine': 17, 'Biology': 2, 'Psychology': 21, 'Art': 1, 'Mathematics': 16, 'Engineering': 8}


In [46]:
from datasets import concatenate_datasets

def pick_balanced_dataset(dataset, num_papers_to_pick, num_labels):
  '''
    Function to pick a sample fraction of a dataset, with papers from each category 
    selected in the same proportion as the original.
  '''
  num_total_papers = len(dataset)
  bal_dict = dict()  

  for label in range(num_labels):
    fos_subj = dataset.filter(lambda example: label in example['labels'])
    subj_count = len(fos_subj)
    small_fos_subj = fos_subj.shuffle(seed=42).select(range(round(subj_count/num_total_papers*num_papers_to_pick)))
    temp_dict = dict()

    temp_dict['data'] = small_fos_subj
    temp_dict['count'] = len(small_fos_subj)
    bal_dict[label] = temp_dict  
 
  print("Total papers picked : {}\n".format(num_papers_to_pick))

  bal_dataset = list()
  for label_text in labels_dict.keys():
    label = labels_dict[label_text]
    num_papers = bal_dict[label]['count']
    print("{} : {} ({:.2f}%)".format(label_text, num_papers, num_papers/num_papers_to_pick*100))
    bal_dataset.append(bal_dict[label]['data'])

  # Finally, concatenate datasets for each subject and then shuffle the contents
  bal_dataset = concatenate_datasets(bal_dataset).shuffle(seed=40)  
  return bal_dataset

In [47]:
fos_train = fos['train']
fos_eval = fos['evaluation']
num_train_papers_to_pick = 20000
num_eval_papers_to_pick = 5000

# Pick sample, balanced train and val datasets
fos_small_train = pick_balanced_dataset(fos_train, num_train_papers_to_pick, num_labels)
fos_small_eval = pick_balanced_dataset(fos_eval, num_eval_papers_to_pick, num_labels)

Total papers picked : 20000

Linguistics : 68 (0.34%)
History : 1590 (7.95%)
Chemistry : 1529 (7.65%)
Philosophy : 266 (1.33%)
Physics : 2628 (13.14%)
Geography : 367 (1.84%)
Business : 309 (1.54%)
Sociology : 238 (1.19%)
Economics : 842 (4.21%)
Geology : 439 (2.20%)
Education : 1112 (5.56%)
Law : 209 (1.04%)
Computer science : 1329 (6.64%)
Environmental science : 1979 (9.89%)
Agricultural and Food sciences : 732 (3.66%)
Materials science : 878 (4.39%)
Political science : 928 (4.64%)
Medicine : 2569 (12.85%)
Biology : 2031 (10.15%)
Psychology : 1093 (5.46%)
Art : 120 (0.60%)
Mathematics : 1300 (6.50%)
Engineering : 1332 (6.66%)


Total papers picked : 5000

Linguistics : 18 (0.36%)
History : 397 (7.94%)
Chemistry : 382 (7.64%)
Philosophy : 67 (1.34%)
Physics : 654 (13.08%)
Geography : 90 (1.80%)
Business : 78 (1.56%)
Sociology : 62 (1.24%)
Economics : 210 (4.20%)
Geology : 111 (2.22%)
Education : 278 (5.56%)
Law : 53 (1.06%)
Computer science : 332 (6.64%)
Environmental science : 493 (9.86%)
Agricultural and Food sciences : 184 (3.68%)
Materials science : 220 (4.40%)
Political science : 232 (4.64%)
Medicine : 641 (12.82%)
Biology : 508 (10.16%)
Psychology : 273 (5.46%)
Art : 32 (0.64%)
Mathematics : 324 (6.48%)
Engineering : 332 (6.64%)


In [50]:
fos_small_train[3]

{'doc_id': '94682184',
 'corpus_id': 94682184,
 'title': 'PREPARATION OF CHITOSAN STABILIZED OFLOXACIN- GOLD NANO CONJUGATE FOR THE IMPROVED ANTI BACTERIAL ACTIVITY AGAINST HUMAN PATHOGENIC BACTERIA',
 'abstract': 'Objective .In the present study, biocom[patible polymer chitosan stabilized ofloxacin- gold nanoconjugate was synthesized for the improved anti bacterial activity against clinical isolate of E.coli and Staphylococcus aureus Methods. Chitosan stabilized oflaxacin-gold nanoparticles conjugate was prepared with aqueous solution of ofloxacin ,gold nanoparticles suspension and chitosan under optimum condition. Synthesized nanoconjugate was .characterized by scanning electron microscopy, FTIR and the synthesized nanoconjugate was tested against E.coli and Staph.aureus adopting well diffusion assay. Result. Synthesis of chitosan stabilized ofloxacin- gold nanoconjugate was primarly confirmed by colour change of the reaction mixture, characteristic change in the FTIR pattern and siz

In [51]:
from transformers import AutoTokenizer
from datasets import Dataset

max_length = 400

def preprocess(dataset, tokenizer):
    '''
      Function to preprocess dataset in the correct format.
    '''
    
    dataset_ = dict()
    labels, texts = list(), list()

    for paper_dict in dataset:
        text = paper_dict['title'] + ' ' + paper_dict['abstract']
        for label in paper_dict['labels']:      # For each label of a paper, add a separate record to the dataset
            labels.append(label)
            texts.append(text)

    dataset_['label'] = labels
    dataset_['text'] = texts
    preprocessed_dataset = Dataset.from_dict(dataset_)

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding=True, truncation=True, max_length=max_length)

    tokenized_dataset = preprocessed_dataset.map(tokenize_function, batched=True) 
    return tokenized_dataset

In [52]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

train = preprocess(fos_small_train, tokenizer)
eval = preprocess(fos_small_eval, tokenizer)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file https://hug

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

### Importing the Model

In [53]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=num_labels)

loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    

In [ ]:
# # JUST A CODE CELL FOR INFINITE RUNNING :)
# !python -m pip install huggingface_hub
# !huggingface-cli login

### Training the model

In [55]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [56]:
import numpy as np

def compute_metrics(pred_eval):
    '''
      Function to compute evaluation metrics like
      cs_precision and cs_recall.
    '''
    logits, labels = pred_eval
    predictions = np.argmax(logits, axis=-1)

    cs_label = labels_dict['Computer science']

    num_total_papers = len(labels)
    num_cs_papers_actual = sum(np.array(labels == cs_label))
    num_cs_papers_pred = sum(np.array(predictions == cs_label))
    num_cs_papers_correct = sum(np.array((predictions == labels) & (labels == cs_label)))

    print("Total papers : {}".format(num_total_papers))
    print("Total CS papers : {}".format(num_cs_papers_actual))
    print("Total CS papers predicted : {}".format(num_cs_papers_pred))
    print("Total papers correctly predicted : {}".format(num_cs_papers_correct))

    cs_precision = num_cs_papers_correct / num_cs_papers_pred
    cs_recall = num_cs_papers_correct / num_cs_papers_actual

    return {"cs_precision": cs_precision, "cs_recall": cs_recall}

### **FOR DEBUGGING ONLY !!**

In [39]:
# Tiny sample for DEBUGGING ONLY
fos_tiny_train = pick_balanced_dataset(fos_train, 500, num_labels)
fos_tiny_eval = pick_balanced_dataset(fos_eval, 500, num_labels)

Total papers picked : 500

Linguistics : 2 (0.40%)
History : 40 (8.00%)
Chemistry : 38 (7.60%)
Philosophy : 7 (1.40%)
Physics : 66 (13.20%)
Geography : 9 (1.80%)
Business : 8 (1.60%)
Sociology : 6 (1.20%)
Economics : 21 (4.20%)
Geology : 11 (2.20%)
Education : 28 (5.60%)
Law : 5 (1.00%)
Computer science : 33 (6.60%)
Environmental science : 49 (9.80%)
Agricultural and Food sciences : 18 (3.60%)
Materials science : 22 (4.40%)
Political science : 23 (4.60%)
Medicine : 64 (12.80%)
Biology : 51 (10.20%)
Psychology : 27 (5.40%)
Art : 3 (0.60%)
Mathematics : 32 (6.40%)
Engineering : 33 (6.60%)


Total papers picked : 500

Linguistics : 2 (0.40%)
History : 40 (8.00%)
Chemistry : 38 (7.60%)
Philosophy : 7 (1.40%)
Physics : 65 (13.00%)
Geography : 9 (1.80%)
Business : 8 (1.60%)
Sociology : 6 (1.20%)
Economics : 21 (4.20%)
Geology : 11 (2.20%)
Education : 28 (5.60%)
Law : 5 (1.00%)
Computer science : 33 (6.60%)
Environmental science : 49 (9.80%)
Agricultural and Food sciences : 18 (3.60%)
Materials science : 22 (4.40%)
Political science : 23 (4.60%)
Medicine : 64 (12.80%)
Biology : 51 (10.20%)
Psychology : 27 (5.40%)
Art : 3 (0.60%)
Mathematics : 32 (6.40%)
Engineering : 33 (6.60%)


In [40]:
fos_tiny_train

Dataset({
    features: ['doc_id', 'corpus_id', 'title', 'abstract', 'labels', 'labels_text'],
    num_rows: 596
})

In [41]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

tiny_train = preprocess(fos_tiny_train, tokenizer)
tiny_eval = preprocess(fos_tiny_eval, tokenizer)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file https://hug

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
tiny_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tiny_train,
    eval_dataset=tiny_eval,
    compute_metrics=compute_metrics,
)

tiny_trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 810
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 306


Epoch,Training Loss,Validation Loss,Cs Precision,Cs Recall
1,No log,1.519404,0.620000,0.688889
2,No log,1.406410,0.561404,0.711111
3,No log,1.322418,0.615385,0.711111


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 828
  Batch size = 8


Total papers : 828
Total CS papers : 45
Total CS papers predicted : 50
Total papers correctly predicted : 31


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 828
  Batch size = 8


Total papers : 828
Total CS papers : 45
Total CS papers predicted : 57
Total papers correctly predicted : 32


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 828
  Batch size = 8


Total papers : 828
Total CS papers : 45
Total CS papers predicted : 52
Total papers correctly predicted : 32




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=306, training_loss=0.8982162974239175, metrics={'train_runtime': 282.2882, 'train_samples_per_second': 8.608, 'train_steps_per_second': 1.084, 'total_flos': 499594075128000.0, 'train_loss': 0.8982162974239175, 'epoch': 3.0})

In [44]:
tiny_evaluation = tiny_trainer.evaluate()
print(tiny_evaluation)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 828
  Batch size = 8


Total papers : 828
Total CS papers : 45
Total CS papers predicted : 52
Total papers correctly predicted : 32
{'eval_loss': 1.322418212890625, 'eval_cs_precision': 0.6153846153846154, 'eval_cs_recall': 0.7111111111111111, 'eval_runtime': 24.9544, 'eval_samples_per_second': 33.18, 'eval_steps_per_second': 4.168, 'epoch': 3.0}


### Training on actual dataset

In [58]:
# Whole dataset will take 52 hours for training on GPU, 20k random samples around 2 hr
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=compute_metrics,
)

In [59]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 32152
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12057


Epoch,Training Loss,Validation Loss,Cs Precision,Cs Recall
1,0.965800,0.948198,0.745614,0.395349
2,0.732400,0.900670,0.713311,0.486047
3,0.522600,0.881514,0.630480,0.702326


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

Total papers : 8077
Total CS papers : 430
Total CS papers predicted : 228
Total papers correctly predicted : 170


Saving model checkpoint to test_trainer/checkpoint-4500
Configuration saved in test_trainer/checkpoint-4500/config.json
Model weights saved in test_trainer/checkpoint-4500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-5000
Configuration saved in test_trainer/checkpoint-5000/config.json
Model weights saved in test_trainer/checkpoint-5000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-5500
Configuration saved in test_trainer/checkpoint-5500/config.json
Model weights saved in test_trainer/checkpoint-5500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-6000
Configuration saved in test_trainer/checkpoint-6000/config.json
Model weights saved in test_trainer/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-6500
Configuration saved in test_trainer/checkpoint-6500/config.json
Model weights saved in test_trainer/checkpoint-6500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint

Total papers : 8077
Total CS papers : 430
Total CS papers predicted : 293
Total papers correctly predicted : 209


Saving model checkpoint to test_trainer/checkpoint-8500
Configuration saved in test_trainer/checkpoint-8500/config.json
Model weights saved in test_trainer/checkpoint-8500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-9000
Configuration saved in test_trainer/checkpoint-9000/config.json
Model weights saved in test_trainer/checkpoint-9000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-9500
Configuration saved in test_trainer/checkpoint-9500/config.json
Model weights saved in test_trainer/checkpoint-9500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-10000
Configuration saved in test_trainer/checkpoint-10000/config.json
Model weights saved in test_trainer/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-10500
Configuration saved in test_trainer/checkpoint-10500/config.json
Model weights saved in test_trainer/checkpoint-10500/pytorch_model.bin
Saving model checkpoint to test_trainer/chec

Total papers : 8077
Total CS papers : 430
Total CS papers predicted : 479
Total papers correctly predicted : 302




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=12057, training_loss=0.8265603963955229, metrics={'train_runtime': 9054.9024, 'train_samples_per_second': 10.652, 'train_steps_per_second': 1.332, 'total_flos': 1.98308008685376e+16, 'train_loss': 0.8265603963955229, 'epoch': 3.0})

In [60]:
evaluation = trainer.evaluate()
print(evaluation)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8077
  Batch size = 8


Total papers : 8077
Total CS papers : 430
Total CS papers predicted : 479
Total papers correctly predicted : 302
{'eval_loss': 0.8815141916275024, 'eval_cs_precision': 0.6304801670146137, 'eval_cs_recall': 0.7023255813953488, 'eval_runtime': 245.1729, 'eval_samples_per_second': 32.944, 'eval_steps_per_second': 4.12, 'epoch': 3.0}


### Saving the Model

In [ ]:
# Save the files that can be directly loaded for model inference
trainer.save_model("/content/scirepeval/final_checkpoint/")

Saving model checkpoint to /content/scirepeval/final_checkpoint/
Configuration saved in /content/scirepeval/final_checkpoint/config.json
Model weights saved in /content/scirepeval/final_checkpoint/pytorch_model.bin


In [ ]:
# Zip the final checkpoint
!zip -r /content/scirepeval/final_checkpoint.zip /content/scirepeval/final_checkpoint/ 

  adding: content/scirepeval/final_checkpoint/ (stored 0%)
  adding: content/scirepeval/final_checkpoint/pytorch_model.bin (deflated 7%)
  adding: content/scirepeval/final_checkpoint/config.json (deflated 64%)
  adding: content/scirepeval/final_checkpoint/training_args.bin (deflated 48%)


In [ ]:
# Download the checkpoint
from google.colab import files
files.download('/content/scirepeval/final_checkpoint.zip')

In [ ]:
# Or, add it to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/scirepeval/final_checkpoint_small.zip '/content/gdrive/MyDrive/G25_IR_TermProject'

## INFERENCE

### Loading pretrained model 


> Install `gdown` using the following command, it will be needed to install files from Google Drive.

In [4]:
!pip install gdown # Needed to install files from Google Drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


> Now use `gdown` to download the classifier model checkpoints and extract the checkpoint to the correct location.

In [5]:
!gdown https://drive.google.com/uc?id=1yGO03wbwVPyGe1HeSaQlYklXBgm0L0WG  

Downloading...
From: https://drive.google.com/uc?id=1yGO03wbwVPyGe1HeSaQlYklXBgm0L0WG
To: /content/scirepeval/classifier_checkpoint.zip
100% 408M/408M [00:03<00:00, 116MB/s]


In [6]:
!unzip classifier_checkpoint.zip -d /content/scirepeval/

Archive:  classifier_checkpoint.zip
   creating: /content/scirepeval/classifier_checkpoint/
  inflating: /content/scirepeval/classifier_checkpoint/pytorch_model.bin  
  inflating: /content/scirepeval/classifier_checkpoint/config.json  
  inflating: /content/scirepeval/classifier_checkpoint/training_args.bin  


> You can now load the pretrained model and model config files from the checkpoint.

**NOTE** : You might need to run the following cell twice 

In [8]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the configuration from the config.json file
pretrained_config = AutoConfig.from_pretrained("classifier_checkpoint/config.json")

# Load the model from the pytorch_model.bin file
pretrained_model = AutoModelForSequenceClassification.from_pretrained("classifier_checkpoint/pytorch_model.bin", config=pretrained_config)

In [ ]:
import numpy as np

def compute_accuracy(pred_eval):
    '''
      Function to compute evaluation metrics like
      overall accuracy, cs-specific and non-cs accuracy.
    '''
    logits, labels = pred_eval
    predictions = np.argmax(logits, axis=-1)

    # Compute the accuracy
    accuracy = np.mean(predictions == labels)
    cs_label = labels_dict['Computer science']

    num_total_papers = len(labels)
    num_cs_papers = sum(np.array(labels == cs_label))
    num_non_cs_papers = num_total_papers - num_cs_papers

    cs_accuracy = np.mean((labels == cs_label) & (predictions == labels)) * (num_total_papers / num_cs_papers)
    non_cs_accuracy = np.mean((labels != cs_label) & (predictions == labels)) * (num_total_papers / num_non_cs_papers)

    return {"accuracy": accuracy, "cs_accuracy": cs_accuracy, "non_cs_accuracy": non_cs_accuracy}

In [ ]:
trainer = Trainer(
    model = pretrained_model,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_accuracy,
)

In [ ]:
# Create a Trainer instance with the loaded model and training arguments
trainer = Trainer(
    model=pretrained_model
)

### Single-input inference

In [ ]:
# Trying out inference on a single prompt
import torch
from transformers import AutoModel, AutoTokenizer, PretrainedConfig, Trainer

# Original category : Biology (Label = 2)
text = "A method for the ultrastructural examination of cell monolayers cultured in plastic microtitre plates. A method is presented by which cells growing as monolayers cultured in microtitre plates can be embedded for electron microscopy. The technique has the following advantages: numerous specimens may be prepared with relatively small numbers of cells, cell-cell interactions remain undisturbed and may be enumerated, and morphology can be studied under circumstances identical with those used in cell-mediated cytotoxicity assays. "

# Load the pretrained SciBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
outputs = trainer.model(**encoding)

logits = outputs.logits
pred = torch.argmax(logits, axis = 1)[0].int().item() # Label = 2 stands for Biology

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp44m4zkd1


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
creating metadata file for /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

storing https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
creating metadata file for /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/33593020f507d72099bd84ea6cd2296feb424fecd62d4a8edcc2a02899af6e29.38339d84e6e392addd730fd85fae32652c4cc7c5423633d6fa73e5f7937bbc38
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingf

In [ ]:
pred # Actual label = 2 (Biology)

2

### Loading the `high_influence_cite` dataset 

In [ ]:
import datasets

high_inf_cite = datasets.load_dataset('allenai/scirepeval', 'high_influence_cite')

{'train': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/high_influence_cite/train.jsonl', 'val': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/train/high_influence_cite/val.jsonl', 'test': 'https://ai2-s2-research-public.s3.us-west-2.amazonaws.com/scirepeval/test/high_influence_cite/meta.jsonl'}


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating evaluation split:   0%|          | 0/1199 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/58626 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7356 [00:00<?, ? examples/s]

Dataset scirepeval downloaded and prepared to /root/.cache/huggingface/datasets/allenai___scirepeval/high_influence_cite/1.1.0/05b6b341c1750891a5539df5b3eb892babd18e8f299bfaba4735b87e20f6cf83. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
high_inf_cite

DatasetDict({
    evaluation: Dataset({
        features: ['query', 'candidates'],
        num_rows: 1199
    })
    train: Dataset({
        features: ['query', 'candidates'],
        num_rows: 58626
    })
    validation: Dataset({
        features: ['query', 'candidates'],
        num_rows: 7356
    })
})

In [ ]:
hic_train = high_inf_cite["train"]
hic_train[3]['query']

{'doc_id': '41825099',
 'title': 'Exact SER Analysis of DFrFT-Based QPSK OFDM System over Frequency Selective Rayleigh Fading Channel with CFO',
 'abstract': 'We present the exact symbol error rate (SER) expression for quadrature phase shift keying (QPSK) modulation in frequency selective Rayleigh fading channel for discrete fractional Fourier transform- (DFrFT-) based orthogonal frequency division multiplexing (OFDM) system in the presence of carrier frequency offset (CFO). The theoretical result is confirmed by means of Monte Carlo simulations. It is shown that the performance of the proposed system, at different values of the DFrFT angle parameter źź,ź is better than that of OFDM based on discrete Fourier transform.',
 'corpus_id': 41825099}

In [ ]:
(hic_train[3]['candidates'])[0]

{'doc_id': '14203403',
 'title': 'Fractional Fourier transform as a signal processing tool: An overview of recent developments',
 'abstract': 'Fractional Fourier transform (FRFT) is a generalization of the Fourier transform, rediscovered many times over the past 100 years. In this paper, we provide an overview of recent contributions pertaining to the FRFT. Specifically, the paper is geared toward signal processing practitioners by emphasizing the practical digital realizations and applications of the FRFT. It discusses three major topics. First, the manuscripts relates the FRFT to other mathematical transforms. Second, it discusses various approaches for practical realizations of the FRFT. Third, we overview the practical applications of the FRFT. From these discussions, we can clearly state that the FRFT is closely related to other mathematical transforms, such as time-frequency and linear canonical transforms. Nevertheless, we still feel that major contributions are expected in the 

> Now, we classify the papers in the `high_influence_cite` subpart of the SciRepEval dataset using the pretrained SciBERT multi-label classifier.

In [ ]:
max_length = 400

def classify(example):
    '''
        Function to classify paper text + abstract into different domains 
        using pretrained classifier.
    '''
    # queries = examples['query']
    
    # for query in queries:
    #     print(query)
    #     break

    # titles = [query['title'] for query in queries]
    # abstracts = [query['abstract'] for query in queries]
    # texts = [title + ' ' + abstract for title, abstract in zip(titles, abstracts)]

    # encodings = tokenizer(text, max_length = max_length, truncation = True, return_tensors="pt")
    # encodings = {k: v.to(trainer.model.device) for k,v in encodings.items()}
    # outputs = trainer.model(**encodings)

    title = example['query']['title']
    abstract = example['query']['abstract']
    text = title + ' ' + abstract

    encoding = tokenizer(text, max_length = max_length, truncation = True, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    output = trainer.model(**encoding)

    logits = output.logits
    example['label'] = torch.argmax(logits, axis = 1)[0].int().item()

    return example

### **Ignore the next few cells, for DEBUGGING PURPOSES ONLY !!**

In [ ]:
hic_train_tiny = hic_train.shuffle(seed=42).select(range(100))

In [ ]:
hic_train_tiny_clf = hic_train_tiny.map(classify)

  0%|          | 0/100 [00:00<?, ?ex/s]

In [ ]:
hic_train_tiny_clf

Dataset({
    features: ['query', 'candidates', 'label'],
    num_rows: 100
})

In [ ]:
cs_label = 5

In [ ]:
hic_train_tiny_filter = hic_train_tiny_clf.filter(lambda example: example['label'] == cs_label)
hic_train_tiny_filter

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['query', 'candidates', 'label'],
    num_rows: 12
})

In [ ]:
hic_train_tiny_filter['query']

[{'doc_id': '5969778',
  'title': 'Improving the Scalability of Data Center Networks with Traffic-aware Virtual Machine Placement',
  'abstract': 'The scalability of modern data centers has become a practical concern and has attracted significant attention in recent years. In contrast to existing solutions that require changes in the network architecture and the routing protocols, this paper proposes using traffic-aware virtual machine (VM) placement to improve the network scalability. By optimizing the placement of VMs on host machines, traffic patterns among VMs can be better aligned with the communication distance between them, e.g. VMs with large mutual bandwidth usage are assigned to host machines in close proximity. We formulate the VM placement as an optimization problem and prove its hardness. We design a two-tier approximate algorithm that efficiently solves the VM placement problem for very large problem sizes. Given the significant difference in the traffic patterns seen in 

In [ ]:
hic_train_tiny_cs = hic_train_tiny_filter.remove_columns('label')

In [ ]:
hic_train_tiny_cs

Dataset({
    features: ['query', 'candidates'],
    num_rows: 12
})

In [ ]:
hic_train_tiny_cs[3]

{'query': {'doc_id': '5678393',
  'title': 'HDR-ARtiSt: High dynamic range advanced real-time imaging system',
  'abstract': 'This paper describes the HDR-ARtiSt hardware platform, a FPGA-based architecture that can produce a real-time high dynamic range video from successive image acquisition. The hardware platform is built around a standard low dynamic range (LDR) CMOS sensor and a Virtex 5 FPGA board. The CMOS sensor is a EV76C560 provided by e2v. This 1.3 Megapixel device offers novel pixel integration/readout modes and embedded image pre-processing capabilities including multiframe acquisition with various exposure times. Our approach consists of a hardware architecture with different algorithms: double exposure control during image capture, building of an HDR image by combining the multiple frames, and final tone mapping for viewing on a LCD display. Our video camera system is able to achieve a real-time video rate of 30 frames per second for a full sensor resolution of 1,280 × 1

In [ ]:
hic_train_tiny_cs

Dataset({
    features: ['query', 'candidates'],
    num_rows: 12
})

In [ ]:
hic_train_tiny_cs.save_to_disk('hic_cs_papers')

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import shutil
shutil.make_archive("hic_cs_papers", "zip", "hic_cs_papers")

'/content/scirepeval/hic_cs_papers.zip'

In [ ]:
# Download the zipped dataset
from google.colab import files
files.download("hic_cs_papers.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -rf hic_cs_papers

### Classifying papers in the `high_influence_cite` dataset

In [ ]:
cs_label = 5  # Actual label for Computer science domain papers in 'Field of Study' dataset

In [ ]:
hic_train_clf = hic_train.map(classify)

  0%|          | 0/58626 [00:00<?, ?ex/s]

In [ ]:
hic_train_filter = hic_train_clf.filter(lambda example: example['label'] == cs_label)
hic_train_cs = hic_train_filter.remove_columns('label')

  0%|          | 0/59 [00:00<?, ?ba/s]

In [ ]:
hic_train_cs.save_to_disk("hic_cs_papers")

Flattening the indices:   0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
# Zip the dataset directory
import shutil
shutil.make_archive("hic_cs_papers", "zip", "hic_cs_papers")

'/content/scirepeval/hic_cs_papers.zip'

In [ ]:
# Add it to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp /content/scirepeval/hic_cs_papers.zip '/content/gdrive/MyDrive/G25_IR_TermProject'

In [ ]:
# Or, download the zipped dataset
from google.colab import files
files.download("hic_cs_papers.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Load dataset from Google Drive

> Install `gdown` using the following command, it will be needed to install files from Google Drive.

In [ ]:
!pip install gdown # Needed to install files from Google Drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


> Now use `gdown` to download the CS-classified papers and extract the file to the correct location.

In [ ]:
!gdown https://drive.google.com/uc?id=1b0g06yMQCjOoO0VaRAs3fNk5Kvqni8Ka  

Downloading...
From: https://drive.google.com/uc?id=1b0g06yMQCjOoO0VaRAs3fNk5Kvqni8Ka
To: /content/scirepeval/hic_cs_papers.zip
100% 56.0M/56.0M [00:00<00:00, 264MB/s]


In [ ]:
!unzip hic_cs_papers.zip -d /content/scirepeval/

Archive:  hic_cs_papers.zip
  inflating: /content/scirepeval/dataset.arrow  
  inflating: /content/scirepeval/dataset_info.json  
  inflating: /content/scirepeval/state.json  


In [ ]:
# To load the dataset back from disk
from datasets import load_from_disk

dataset = load_from_disk('hic_cs_papers')

In [ ]:
dataset

Dataset({
    features: ['query', 'candidates'],
    num_rows: 6165
})

In [ ]:
dataset[3]

{'query': {'doc_id': '8202993',
  'title': 'Coherent Filtering: Detecting Coherent Motions from Crowd Clutters',
  'abstract': 'Coherent motions, which describe the collective movements of individuals in crowd, widely exist in physical and biological systems. Understanding their underlying priors and detecting various coherent motion patterns from background clutters have both scientific values and a wide range of practical applications, especially for crowd motion analysis. In this paper, we propose and study a prior of coherent motion called Coherent Neighbor Invariance, which characterizes the local spatiotemporal relationships of individuals in coherent motion. Based on the coherent neighbor invariance, a general technique of detecting coherent motion patterns from noisy time-series data called Coherent Filtering is proposed. It can be effectively applied to data with different distributions at different scales in various real-world problems, where the environments could be sparse 